<a href="https://colab.research.google.com/github/oonya/MyPckMachineLearning/blob/main/UseI2v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install illustration2vec

  Created wheel for illustration2vec: filename=illustration2vec-0.1.0-cp36-none-any.whl size=9645 sha256=4b51f115ad91404255a05aae065cd6e65fc8cb66d0aed2b781a6e44ea5314c87
  Stored in directory: /root/.cache/pip/wheels/15/ab/0d/3f40e0e6ed336a058cd7a3fb897a998ca85274039d5167d1e5
Successfully built illustration2vec


In [2]:
import i2v
from PIL import Image

path = 'drive/My Drive/illustration2vec/'

# In the feature vector extraction, you do not need to specify the tag.
illust2vec = i2v.make_i2v_with_chainer(path+"illust2vec_ver200.caffemodel")

In [48]:
img = Image.open(path+"images/miku.jpg")

# extract a 4,096-dimensional feature vector
result_real = illust2vec.extract_feature([img])
print("shape: {}, dtype: {}".format(result_real.shape, result_real.dtype))
print(result_real)


type([np.uint8([[1, 2, 3]]), 0])

shape: (1, 4096), dtype: float32
[[7.6607313 4.6432104 0.8488899 ... 0.3083177 1.641011  7.47949  ]]


list

In [ ]:
# 今はdriveから読み込んでいるので必要ない

%%bash

# git clone https://github.com/rezoo/illustration2vec.git
# sh illustration2vec/get_models.sh
# pip install illustration2vec


# mv illustration2vec/ drive/My\ Drive
# ls drive/My\ Drive/illustration2vec

In [3]:
# データセットを作る
# iv2でmodelを読み込み済み(重いからセルを分けただけ)

import os
from PIL import Image
trainpath = 'drive/My Drive/picturesForMachineLearning/train/'
animal_path = 'animal_img/'
person_path = 'person_img/'


cnt = 1


train_data = []
for filename in os.listdir(trainpath + person_path):
  img = Image.open(trainpath + person_path + filename)
  result_real = illust2vec.extract_feature([img])

  train_data.append([result_real, 0])

  cnt += 1
  # if cnt > 5:
  #   break

cnt = 1

for filename in os.listdir(trainpath + animal_path):
  try:
    img = Image.open(trainpath + animal_path + filename)
    result_real = illust2vec.extract_feature([img])

    train_data.append([result_real, 1])
  except Exception as e:
    print(e, filename)
    pass

  cnt += 1
  # if cnt > 5:
  #   break




# [np.uint8([[1, 2, 3]]), 0]

# train_data

operands could not be broadcast together with shapes (3,) (2,)  g.png
operands could not be broadcast together with shapes (3,) (2,)  e12.png
operands could not be broadcast together with shapes (3,) (2,)  e (4).png
operands could not be broadcast together with shapes (3,) (2,)  shirokuma-shirokuro-400x400.png


In [16]:
# test用のデータ
# 名前はvalidになってるけど、それはごめん
# iv2でmodelを読み込み済み(重いからセルを分けただけ)

import os
from PIL import Image
validpath = 'drive/My Drive/picturesForMachineLearning/valid/'
animal_path = 'animal_img/'
person_path = 'person_img/'


cnt = 1


train_data = []
for filename in os.listdir(validpath + person_path):
  img = Image.open(validpath + person_path + filename)
  result_real = illust2vec.extract_feature([img])

  train_data.append([result_real, 0])

  cnt += 1
  # if cnt > 5:
  #   break

cnt = 1

for filename in os.listdir(validpath + animal_path):
  try:
    img = Image.open(validpath + animal_path + filename)
    result_real = illust2vec.extract_feature([img])

    train_data.append([result_real, 1])
  except Exception as e:
    print(e, filename)
    pass

  cnt += 1
  # if cnt > 5:
  #   break





# train_data

operands could not be broadcast together with shapes (3,) (2,)  18.png


In [19]:
# to make dataset takes 2 minutes

import numpy as np
rng = np.random.default_rng()
r_a = rng.permutation(train_data)

valid_vector = r_a[:, 0]
valid_vector = np.concatenate(valid_vector)
valid_label = r_a[:, 1]
valid_label=np.array(valid_label,dtype='float32')

print(valid_vector.shape)
print(valid_label.shape)

np.save('drive/My Drive/pck_machineLearning/valid_vector', valid_vector)
np.save('drive/My Drive/pck_machineLearning/valid_label', valid_label)

(59, 4096)
(59,)


In [4]:
# to make dataset takes 2 minutes

import numpy as np
rng = np.random.default_rng()
r_a = rng.permutation(train_data)

vector = r_a[:, 0]
vector = np.concatenate(vector)
label = r_a[:, 1]
label=np.array(label,dtype='float32')


a = np.array([0, 1], np.float32)
# print(a)
# print(vector.shape)

print(vector.shape)
print(label.shape)

(236, 4096)
(236,)


In [15]:
np.save('drive/My Drive/pck_machineLearning/vecotrArray', vector)
np.save('drive/My Drive/pck_machineLearning/labelArray', label)

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation
import tensorflow.keras

N_CATEGORIES = 1

model = Sequential()

model.add(Dense(units=32, activation='relu',input_shape=(4096,)))
model.add(Dense(units=N_CATEGORIES, activation='sigmoid'))


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# print(model.summary())
model.fit(vector, label, epochs=30, batch_size=32,validation_split=0.1)


path = 'drive/My Drive/pck_machineLearning'
model.save(filepath='drive/My Drive/pck_machineLearning/pck_model_activationSigmoid.h5', save_format='h5')


Epoch 1/30
7/7 [==============================] - 0s 20ms/step - loss: 0.8321 - accuracy: 0.7689 - val_loss: 0.2954 - val_accuracy: 0.8750
Epoch 2/30
7/7 [==============================] - 0s 5ms/step - loss: 0.2404 - accuracy: 0.9104 - val_loss: 0.2572 - val_accuracy: 0.9167
Epoch 3/30
7/7 [==============================] - 0s 5ms/step - loss: 0.1847 - accuracy: 0.9245 - val_loss: 0.0690 - val_accuracy: 0.9583
Epoch 4/30
7/7 [==============================] - 0s 5ms/step - loss: 0.1320 - accuracy: 0.9387 - val_loss: 0.0468 - val_accuracy: 0.9583
Epoch 5/30
7/7 [==============================] - 0s 5ms/step - loss: 0.1063 - accuracy: 0.9623 - val_loss: 0.0385 - val_accuracy: 1.0000
Epoch 6/30
7/7 [==============================] - 0s 5ms/step - loss: 0.0982 - accuracy: 0.9623 - val_loss: 0.0603 - val_accuracy: 0.9583
Epoch 7/30
7/7 [==============================] - 0s 5ms/step - loss: 0.0758 - accuracy: 0.9670 - val_loss: 0.0180 - val_accuracy: 1.0000
Epoch 8/30
7/7 [=================

In [22]:
score = model.evaluate(valid_vector, valid_label, verbose=1)
print('正解率=', score[1], 'loss=', score[0])	

2/2 [==============================] - 0s 3ms/step - loss: 0.2107 - accuracy: 0.9492
正解率= 0.9491525292396545 loss= 0.2106798142194748
